In [25]:
pip install polars pyarrow


Note: you may need to restart the kernel to use updated packages.


In [26]:
import h2o
from h2o.automl import H2OAutoML


In [27]:


h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,25 mins 27 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 18 days
H2O_cluster_name:,H2O_from_python_Natasha_pjrph0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.131 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [28]:

# Load data into H2O
train = h2o.import_file('train_data.csv')
test = h2o.import_file('test_data.csv')


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [29]:

# Set target and features
target = 'critical_discharge'
features = train.columns
features.remove(target)


In [30]:
train['critical_discharge'] = train['critical_discharge'].asfactor()
test['critical_discharge'] = test['critical_discharge'].asfactor()


In [31]:
train['diagnosisstring'] = train['diagnosisstring'].asfactor()
test['diagnosisstring'] = test['diagnosisstring'].asfactor()


In [32]:
import h2o
import pandas as pd

# Get the train levels
train_levels = train['diagnosisstring'].levels()[0]

# Convert the test frame to a Pandas DataFrame
test_pd = test.as_data_frame()

# Replace unseen levels with 'other'
test_pd['diagnosisstring'] = test_pd['diagnosisstring'].apply(
    lambda x: x if x in train_levels else "other"
)

# Convert back to an H2OFrame
test = h2o.H2OFrame(test_pd)
test['diagnosisstring'] = test['diagnosisstring'].asfactor()


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [33]:
print(train['diagnosisstring'].levels())
print(test['diagnosisstring'].levels())


[['Unknown', 'burns/trauma|trauma - CNS|intracranial injury|with subarachnoid hemorrhage', 'burns/trauma|trauma - skeletal|bone fracture(s)|lumbar spine', 'burns/trauma|trauma - skeletal|bone fracture(s)|pelvis', 'burns/trauma|trauma - skeletal|bone fracture(s)|ribs', 'burns/trauma|trauma - skeletal|bone fracture(s)|right lower extremity|femur', 'burns/trauma|trauma - skeletal|bone fracture(s)|sternum', 'burns/trauma|trauma- head and neck|facial bone fracture|orbital trauma', 'burns/trauma|trauma- head and neck|facial bone fracture|zygoma fracture', 'cardiovascular|arrhythmias|SVT', 'cardiovascular|arrhythmias|atrial fibrillation', 'cardiovascular|arrhythmias|atrial fibrillation|with rapid ventricular response', 'cardiovascular|arrhythmias|atrial flutter', 'cardiovascular|arrhythmias|atrial flutter|1:2 conduction', 'cardiovascular|arrhythmias|bradycardia', 'cardiovascular|arrhythmias|bradycardia|junctional', 'cardiovascular|arrhythmias|sinus tachycardia', 'cardiovascular|arrhythmias|ve

In [34]:

# Run AutoML
aml = H2OAutoML(max_models=20, seed=42, balance_classes=True)
aml.train(x=features, y=target, training_frame=train)


AutoML progress: |█
20:08:26.945: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_2_20250120_200826_model_3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    227                227                         1.82246e+06            1            15           14.8811       2             576           431.938

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 7.408642426052769e-20
RMSE: 2.721882147715578e-10
LogLoss: 2.052123790058445e-10
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999999996398676
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      626  0    0        (0.0/626.0)
1      0    627  0        (0.0/627.0)
Total  626  627  0        (0.0/1253.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value    idx
---------------------------  -----------  -------  -----
max f1                       1            1        0
max f2                       1            1        0
max f0point5                 1            1        0
max accuracy                 1            1        0
max precision                1            1        0
max recall                   1            1        0
max specificity              1            1        0
max absolute_mcc             1            1        0
max min_per_class_accuracy   1            1        0
max mean_per_class_accuracy  1            1        0
max tns                      1            626      0
max fns                      1            0        0
max fps                      1.77481e-11  626      399
max tps                      1            627      0
max tnr                      1            1        0
max fnr                      1            0        0
max fpr                      1.77481e-11  1        399
max tpr                      1            1        0

Gains/Lift Table: Avg response rate: 50,04 %, avg score: 50,04 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0127694                   1                  1.99841  1.99841            1                1            1                           1                   0.0255183       0.0255183                  99.8405  99.8405            0.0255183
2        0.0215483                   1                  1.99841  1.99841            1                1            1                           1                   0.0175439       0.0430622                  99.8405  99.8405            0.0430622
3        0.0327215                   1                  1.99841  1.99841            1                1            1                           1                   0.0223285       0.0653907                  99.8405  99.8405            0.0653907
4        0.0415004                   1                  1.99841  1.99841            1                1            1                           1                   0.0175439       0.0829346                  99.8405  99.8405            0.0829346
5        0.0526736                   1                  1.99841  1.99841            1                1            1                           1                   0.0223285       0.105263                   99.8405  99.8405            0.105263
6  

In [35]:

# View the leaderboard
lb = aml.leaderboard
print(lb)


model_id                                                auc      logloss    aucpr    mean_per_class_error         rmse          mse
GBM_grid_1_AutoML_2_20250120_200826_model_3               1  0.000210529        1                       0  0.00260444   6.78308e-06
GBM_2_AutoML_2_20250120_200826                            1  6.44068e-16        1                       0  3.62036e-15  1.3107e-29
XRT_1_AutoML_2_20250120_200826                            1  0.273322           1                       0  0.303884     0.0923452
GBM_grid_1_AutoML_2_20250120_200826_model_1               1  0                  1                       0  0            0
DeepLearning_grid_3_AutoML_2_20250120_200826_model_1      1  0.000765366        1                       0  0.00565738   3.20059e-05
GBM_4_AutoML_2_20250120_200826                            1  7.66748e-16        1                       0  3.2722e-15   1.07073e-29
GLM_1_AutoML_2_20250120_200826                            1  0.01278            1        

In [36]:
preds = aml.predict(test)
print(preds)  # Check the output columns


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset column 'diagnosisstring' has levels not trained on: ["other"]
  warnings.warn(w)


  predict           p0           p1
        0  1            2.23941e-11
        0  1            1.67942e-09
        0  1            1.19107e-09
        0  1            4.68031e-11
        0  1            8.06824e-10
        0  1            6.69652e-11
        0  1.31362e-08  1
        0  1            1.30961e-09
        0  1            1.63096e-09
        0  1            3.64066e-11
[200 rows x 3 columns]



In [37]:
from sklearn.metrics import precision_recall_curve, auc

# Get the true labels and predicted probabilities
y_true = test['critical_discharge'].as_data_frame()['critical_discharge']

# Make predictions on test data
preds = aml.predict(test)
y_scores = preds['p1'].as_data_frame(use_multi_thread=True,)['p1']  # Assuming 'p1' is the predicted probability for class 1

# Calculate precision, recall, and AUC-PR
precision, recall, threshold = precision_recall_curve(y_true, y_scores)
auc_pr = auc(recall, precision)
print(f"AUC-PR: {auc_pr}")


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset column 'diagnosisstring' has levels not trained on: ["other"]
  warnings.warn(w)


Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
AUC-PR: 1.0


In [17]:
print(preds.columns)


['predict', 'p0', 'p1']


In [8]:
print(test.columns)


['patientunitstayid', 'age', 'ethnicity', 'hospitaladmitsource', 'hospitaldischargelocation', 'hospitaldischargestatus', 'labresult', 'diagnosisstring', 'icd9code', 'treatmentstring', 'age_group', 'admitdxtext_length', 'is_female', 'critical_discharge']


In [10]:
print(train['critical_discharge'].unique())


  C1
   0
   1
[2 rows x 1 column]



In [11]:
aml.leaderboard


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_1_20250120_172333_model_17,1.39721e-07,1.9522e-14,7.27513e-08,7.05875e-08,1.9522e-14
GBM_grid_1_AutoML_1_20250120_172333_model_19,1.39721e-07,1.9522e-14,7.27513e-08,7.05875e-08,1.9522e-14
GBM_grid_1_AutoML_1_20250120_172333_model_2,1.39721e-07,1.9522e-14,7.27513e-08,7.05875e-08,1.9522e-14
GBM_grid_1_AutoML_1_20250120_172333_model_13,1.39721e-07,1.9522e-14,7.27513e-08,7.05875e-08,1.9522e-14
GBM_grid_1_AutoML_1_20250120_172333_model_1,3.30671e-07,1.09343e-13,8.27802e-08,3.11002e-07,1.09343e-13
GBM_grid_1_AutoML_1_20250120_172333_model_16,0.000634837,4.03018e-07,0.000277034,0.000408196,4.03018e-07
StackedEnsemble_AllModels_1_AutoML_1_20250120_172333,0.00273976,7.50627e-06,0.00156764,0.00172258,7.50627e-06
GLM_1_AutoML_1_20250120_172333,0.0034585,1.19612e-05,0.00285516,0.00282503,1.19612e-05
StackedEnsemble_BestOfFamily_1_AutoML_1_20250120_172333,0.00374225,1.40044e-05,0.00224006,0.00249188,1.40044e-05
GBM_2_AutoML_1_20250120_172333,0.0110038,0.000121084,0.00605146,0.00810294,0.000121084


In [51]:
# Get the best model
best_model = aml.leader


In [52]:

# Evaluate model performance on the test set
perf = best_model.model_performance(test)


In [53]:


# Print key performance metrics
print("AUC:", perf.auc())
print("Logloss:", perf.logloss())
print("Confusion Matrix:")
print(perf.confusion_matrix())

AUC: 1.0
Logloss: 3.459647956244093e-05
Confusion Matrix:
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9931157914848616
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      157  0    0        (0.0/157.0)
1      0    43   0        (0.0/43.0)
Total  157  43   0        (0.0/200.0)


In [55]:


# Find the optimal threshold for the best F1-score
optimal_threshold = perf.find_threshold_by_max_metric("f1")
print(f"Optimal Threshold: {optimal_threshold}")


Optimal Threshold: 0.9931157914848616


In [56]:

# Predict on the test set using the optimal threshold
predictions = best_model.predict(test)
predictions_df = predictions.as_data_frame()


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset column 'diagnosisstring' has levels not trained on: ["other"]
  warnings.warn(w)
c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [57]:

# Adjust predictions based on the optimal threshold
predicted = (predictions_df["p1"] > optimal_threshold).astype(int)


In [58]:

from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the predictions
actual = test['critical_discharge'].as_data_frame()['critical_discharge']  # Replace with your response column name
print("Classification Report:")
print(classification_report(actual, predicted))
print("Confusion Matrix:")
print(confusion_matrix(actual, predicted))


c:\Users\Natasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       157
           1       1.00      0.98      0.99        43

    accuracy                           0.99       200
   macro avg       1.00      0.99      0.99       200
weighted avg       1.00      0.99      0.99       200

Confusion Matrix:
[[157   0]
 [  1  42]]
